In [ ]:
import os
import sys
import subprocess
import time
from datetime import datetime

# ============================================================================
# 1. 配置区域 (Configuration)
# ============================================================================

# 路径配置 (请根据你的实际目录结构调整)
# 假设 Notebook 在项目根目录下，如果不在，请修改 PROJECT_ROOT
PROJECT_ROOT = "." 
TRAIN_SCRIPT = os.path.join(PROJECT_ROOT, "drugreflector_train", "train.py")
EXTRACT_SCRIPT = os.path.join(PROJECT_ROOT, "drugreflector_train", "extract_single_fold_metrics.py")
DATA_FILE = os.path.join(PROJECT_ROOT, "processed_data", "training_data_lincs2020_chemfiltered_1201.pkl")
BASE_OUTPUT_DIR = os.path.join(PROJECT_ROOT, "models", "hyperparam_search_1fold_round2_local")

# 检查文件是否存在
if not os.path.exists(TRAIN_SCRIPT):
    print(f"警告: 找不到训练脚本: {TRAIN_SCRIPT}")
if not os.path.exists(DATA_FILE):
    print(f"警告: 找不到数据文件: {DATA_FILE}")

# 训练通用参数
EPOCHS = 50
BATCH_SIZE = 256
DEVICE = "cuda"  # Windows上通常显式指定cuda，如果没有GPU则改为 "cpu"
NUM_WORKERS = 0  # Windows下多进程Dataloader常有问题，建议设为0或较小值(如2)
FOLD_ID = 0
FOCAL_GAMMA = 2.0
CURRENT_MIN_LR = 1e-6
PLOT_DIR = "training_plots"

# Round 2 任务列表 (Task_ID, LR, Scheduler, WD, Dropout)
# 对应你的 submit_round2_hyperparams.sh
TASKS = [
    {"id": 0, "lr": 0.015, "scheduler": "exponential", "wd": 5e-5, "dropout": 0.5},
    {"id": 1, "lr": 0.015, "scheduler": "exponential", "wd": 5e-5, "dropout": 0.55},
    {"id": 2, "lr": 0.015, "scheduler": "exponential", "wd": 5e-5, "dropout": 0.6},
    {"id": 3, "lr": 0.015, "scheduler": "exponential", "wd": 5e-5, "dropout": 0.64},
    {"id": 4, "lr": 0.02,  "scheduler": "exponential", "wd": 5e-5, "dropout": 0.6},
    {"id": 5, "lr": 0.02,  "scheduler": "exponential", "wd": 5e-5, "dropout": 0.64},
    {"id": 6, "lr": 0.03,  "scheduler": "exponential", "wd": 5e-5, "dropout": 0.64},
    {"id": 7, "lr": 0.05,  "scheduler": "exponential", "wd": 5e-5, "dropout": 0.64},
]

# ============================================================================
# 2. 辅助函数
# ============================================================================

def get_scheduler_params(scheduler_type):
    """根据调度器类型返回衰减参数"""
    if scheduler_type == "step":
        return 0.1, "30 40"
    elif scheduler_type == "exponential":
        return 0.95, None
    else:  # cosine
        return 0.1, None

def run_command(cmd_args, log_file_path):
    """运行系统命令并将输出重定向到文件和控制台"""
    with open(log_file_path, "w", encoding="utf-8") as log_file:
        # 启动进程
        process = subprocess.Popen(
            cmd_args,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
            bufsize=1
        )
        
        # 实时读取输出
        for line in process.stdout:
            print(line, end="")  # 输出到 Notebook
            log_file.write(line) # 输出到日志文件
            
        process.wait()
        return process.returncode

# ============================================================================
# 3. 主循环 (Training Loop)
# ============================================================================

print("==================================================")
print(f"开始执行 Round 2 超参数优化 (Windows 本地版)")
print(f"总任务数: {len(TASKS)}")
print(f"输出目录: {BASE_OUTPUT_DIR}")
print("==================================================\n")

if not os.path.exists(BASE_OUTPUT_DIR):
    os.makedirs(BASE_OUTPUT_DIR)

for task in TASKS:
    task_id = task["id"]
    lr = task["lr"]
    scheduler = task["scheduler"]
    wd = task["wd"]
    dropout = task["dropout"]
    
    # 准备路径和参数
    config_id = f"lr{lr}_sch{scheduler}_wd{wd}_drop{dropout}"
    output_dir = os.path.join(BASE_OUTPUT_DIR, f"config_{config_id}_task{task_id}")
    
    # 创建任务输出目录
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # 获取 LR Decay 参数
    lr_decay_rate, lr_decay_epochs = get_scheduler_params(scheduler)
    
    print(f"Processing Task {task_id}/{len(TASKS)-1}: {config_id}")
    print(f"  > Output: {output_dir}")
    
    # 构建训练命令
    # 注意: Windows下 subprocess 接收列表形式的参数更安全
    cmd = [
        sys.executable, "-u", TRAIN_SCRIPT,
        "--data-file", DATA_FILE,
        "--output-dir", output_dir,
        "--fold", str(FOLD_ID),
        "--epochs", str(EPOCHS),
        "--batch-size", str(BATCH_SIZE),
        "--learning-rate", str(lr),
        "--lr-scheduler", scheduler,
        "--lr-decay-rate", str(lr_decay_rate),
        "--min-lr", str(CURRENT_MIN_LR),
        "--weight-decay", str(wd),
        "--dropout", str(dropout),
        "--focal-gamma", str(FOCAL_GAMMA),
        "--plot-dir", PLOT_DIR,
        "--device", DEVICE,
        "--num-workers", str(NUM_WORKERS),
        "--save-every", "10",
        "--quiet"
    ]
    
    if lr_decay_epochs:
        cmd.extend(["--lr-decay-epochs", lr_decay_epochs])
    
    # 记录开始时间
    start_time = time.time()
    log_file = os.path.join(output_dir, "train_log.txt")
    
    print(f"  > 开始训练... (日志: {log_file})")
    
    # --- 执行训练 ---
    return_code = run_command(cmd, log_file)
    
    end_time = time.time()
    elapsed = end_time - start_time
    
    # 记录配置信息到 config.txt (模仿原脚本行为)
    with open(os.path.join(output_dir, "config.txt"), "w", encoding="utf-8") as f:
        f.write(f"Task ID: {task_id}\n")
        f.write(f"Config: {config_id}\n")
        f.write(f"Status: {'Success' if return_code == 0 else 'Failed'}\n")
        f.write(f"Elapsed: {elapsed:.2f}s\n")
        f.write(f"Params: LR={lr}, Drop={dropout}, WD={wd}, Sch={scheduler}\n")

    if return_code == 0:
        print(f"  ✓ 训练完成 (耗时: {elapsed/60:.1f} min)")
        
        # --- 提取指标 ---
        print("  > 提取指标...")
        extract_cmd = [
            sys.executable, "-u", EXTRACT_SCRIPT,
            "--output-dir", output_dir,
            "--fold-id", str(FOLD_ID)
        ]
        extract_code = run_command(extract_cmd, os.path.join(output_dir, "extract_log.txt"))
        
        if extract_code == 0:
            print("  ✓ 指标提取成功")
        else:
            print("  ✗ 指标提取失败")
            
    else:
        print(f"  ✗ 训练失败 (返回码: {return_code})")
        print("  请检查上方日志或 train_log.txt")

    print("-" * 50)
    # 稍微休息一下，让显卡喘口气
    time.sleep(2)

print("所有任务执行完毕。")

开始执行 Round 2 超参数优化 (Windows 本地版)
总任务数: 8
输出目录: .\models\hyperparam_search_1fold_round2_local

Processing Task 0/7: lr0.015_schexponential_wd5e-05_drop0.5
  > Output: .\models\hyperparam_search_1fold_round2_local\config_lr0.015_schexponential_wd5e-05_drop0.5_task0
  > 开始训练... (日志: .\models\hyperparam_search_1fold_round2_local\config_lr0.015_schexponential_wd5e-05_drop0.5_task0\train_log.txt)

🧬 DRUGREFLECTOR TRAINING
  Based on Science 2025 paper
  Training on LINCS 2020 dataset

📋 Training Configuration:
  Mode: Single fold (0)
  Data file: .\processed_data\training_data_lincs2020_chemfiltered_1201.pkl
  Output dir: .\models\hyperparam_search_1fold_round2_local\config_lr0.015_schexponential_wd5e-05_drop0.5_task0
  Epochs: 50
  Batch size: 256
  Initial LR: 0.015
  Focal γ: 2.0
  Device: cuda
  Workers: 0

📂 Loading Training Data
  Loading from: processed_data\training_data_lincs2020_chemfiltered_1201.pkl
  ✓ Data loaded successfully
  Keys: ['X', 'y', 'folds', 'sample_meta', 'metadata'

In [12]:
import sys, runpy
from pathlib import Path

argv_backup = sys.argv
sys.argv = [
    "train.py",
    "--data-file", "processed_data/training_data_lincs2020_final.pkl",
    "--compound-info", "datasets/LINCS2020/compoundinfo.txt",
    "--fold", "0",
]

# 自动加入脚本所在目录到 sys.path
script_path = Path("drugreflector_twotower/train.py").resolve()
sys.path.append(str(script_path.parent))

runpy.run_path(script_path, run_name="__main__")

sys.argv = argv_backup


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\ProgramData\anaconda3\envs\chemprop\Lib\site-packages\IPython\core\interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\11234\AppData\Local\Temp\ipykernel_26428\3445058657.py", line 16, in <module>
    runpy.run_path(script_path, run_name="__main__")
  File "<frozen runpy>", line 291, in run_path
  File "<frozen runpy>", line 98, in _run_module_code
  File "<frozen runpy>", line 88, in _run_code
  File "E:\科研\Models\drugreflector\drugreflector_twotower\train.py", line 16, in <module>
    from trainer import TwoTowerTrainer
  File "E:\科研\Models\drugreflector\drugreflector_twotower\trainer.py", line 18, in <module>
    from models import TowTowerModel
  File "E:\科研\Models\drugreflector\drugreflector_twotower\models.py", line 13, in <module>
    from chemprop import nn as cnn
  File "d:\ProgramData\anaconda3\envs\chemprop\Lib\site-packages\chemprop\__init__.py", line 1, in <module>

In [3]:
$python train.py \
    --data-file processed_data/training_data_lincs2020_final.pkl \
    --compound-info datasets/LINCS2020/compoundinfo.txt \
    --fold 0

SyntaxError: invalid syntax (3033893754.py, line 1)

In [ ]:
from chemprop import nn as cnn


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [4]:
import torch

torch.cuda.is_available()

True

#!/bin/bash
#SBATCH -J twotower-train
#SBATCH -p gpu_4l
#SBATCH -N 1 
#SBATCH -o ../err_out/twotower_%j.out
#SBATCH -e ../err_out/twotower_%j.err
#SBATCH --no-requeue
#SBATCH -A tangc_g1
#SBATCH --qos=tangcg4c
#SBATCH --gres=gpu:1
#SBATCH --overcommit
#SBATCH --mincpus=30

source /appsnew/source/Tensorflow2.11_Keras2.11.0_keras-nightly2.13.0_torch1.13.1_py3.9.4_cuda11.4.4.sh
unset LD_LIBRARY_PATH

python -u ../drugreflector_twotower/train.py \
    --data-file ../datasets/training_data_lincs2020_final.pkl \
    --compound-info ../datasets/LINCS2020/compoundinfo_beta.txt \
    --output-dir ../models/twotower_ensemble \
    --all-folds \
    --chem-hidden-dim 512 \
    --fusion-method concat \
    --mpnn-depth 3 \
    --epochs 50 \
    --batch-size 256 \
    --learning-rate 0.0139 \
    --use-3d